# Classification


##  0. Preprocessing

In [ ]:
import pandas as pd

In [46]:
# load all the cleanning data
fs= pd.read_csv('all_fed_speeches.csv')
fs = df.iloc[:, [1, -1]]
pm=pd.read_csv('df_minutes.csv')
pm.columns = ['date', 'text']
pc=pd.read_csv('df_press_conferences.csv')
pc.columns = ['date', 'text']

In [49]:
# Concatenate the DataFrames
df = pd.concat([fs, pm, pc], ignore_index=True)
# Sort by the first column
df.sort_values(by=df.columns[0], inplace=True)
# Drop rows with any NA values
df.dropna(inplace=True)
# Optionally, reset the index after dropping rows
df.reset_index(drop=True, inplace=True)

In [50]:
#check the head
df.head()

,date,text
0,2011-01-26,"The Federal Reserve, the central bank of the U..."
1,2011-03-15,"The Federal Reserve, the central bank of the U..."
2,2011-04-27,"The Federal Reserve, the central bank of the U..."
3,2011-06-22,"The Federal Reserve, the central bank of the U..."
4,2011-08-09,"The Federal Reserve, the central bank of the U..."


In [52]:
#check the shape
df.shape

(869, 2)

## 1. Word list

In [63]:
# define my word list
hawkish_words = [
    "inflation", "interest rate hike", "economic overheating", "tightening", 
    "monetary tightening", "inflationary pressures", "rate increase", 
    "fiscal restraint", "price stability", "interest rate rise", "higher interest rates", 
    "excess demand", "inflation expectations", "economic overheating", "rate normalization", 
    "output gap", "asset bubbles", "credit tightening", "wage pressures", 
    "monetary policy normalization", "liquidity constraints", "restrictive policy", 
    "policy tightening", "balance sheet reduction", "bond tapering", 
    "tapering", "rising yields", "inflation control", "monetary contraction", 
    "core inflation", "price hikes", "higher yields", "interest rate ceilings", 
    "currency appreciation", "price growth", "rate adjustment", "debt concerns", 
    "capital flight", "policy reversal", "currency tightening", "price pressures", 
    "overheating risks", "recessionary pressures", "credit constraints", "supply shock", 
    "demand shock", "cost-push inflation", "supply-side constraints", "inflation surge", 
    "labor market overheating", "housing bubble", "credit risk", "overleveraging", 
    "sovereign risk", "deleveraging", "capital outflows", "market overheating", 
    "borrowing costs", "fiscal deficit reduction", "trade deficit", "policy correction", 
    "central bank hawkishness", "debt issuance", "credit squeeze", "inflationary spiral", 
    "bank lending restrictions", "financial instability", "asset repricing", "credit downgrades", 
    "debt tightening", "monetary policy shift", "higher risk premiums", "credit withdrawal", 
    "fiscal adjustment", "inflation targeting", "rate volatility", "real interest rates", 
    "liquidity withdrawal", "rate hikes", "price controls", "yield curves", "tight credit", 
    "policy credibility", "commodity price surge", "import costs", "dollar strengthening", 
    "debt servicing", "credit rating", "reserve requirements", "withdrawal of stimulus", 
    "asset bubbles", "regulatory tightening", "central bank intervention", "currency tightening"
    # Add more terms as needed
]
dovish_words = [
    "lower rates", "stimulus", "economic growth", "accommodative", 
    "quantitative easing", "rate cut", "fiscal stimulus", "monetary expansion", 
    "interest rate cut", "expansionary policy", "liquidity injection", 
    "stimulus package", "growth support", "credit easing", "policy easing", 
    "rate reduction", "fiscal easing", "employment growth", "credit growth", 
    "labor market recovery", "consumer spending", "investment incentives", 
    "growth prospects", "monetary accommodation", "low inflation", 
    "policy accommodation", "supportive measures", "bond purchases", 
    "balance sheet expansion", "inflation tolerance", "interest rate reduction", 
    "output expansion", "currency devaluation", "rate cuts", "stimulus measures", 
    "deficit spending", "unemployment reduction", "credit creation", 
    "negative interest rates", "low yield environment", "easy money", 
    "liquidity support", "quantitative easing measures", "demand-side policies", 
    "supply stimulus", "credit expansion", "soft monetary policy", 
    "supporting growth", "debt issuance", "market stability", "job creation", 
    "consumer price index", "housing support", "asset purchases", 
    "monetary flexibility", "financial support", "fiscal expansion", "rate accommodation", 
    "currency easing", "expansionary monetary policy", "market intervention", 
    "fiscal policy boost", "credit market support", "capital flow management", 
    "fiscal policy adjustment", "liquidity measures", "low rates", "consumer demand", 
    "economic recovery", "inflation tolerance", "supporting liquidity", "employment stimulus", 
    "financial stability", "equity market support", "unemployment stimulus", "budget deficits", 
    "central bank dovishness", "credit facilitation", "stimulus continuation", 
    "low inflation environment", "growth policies", "employment growth", "debt expansion", 
    "investment stimulus", "consumer confidence", "trade growth", "wage growth", 
    "negative interest rates", "policy flexibility", "corporate bond purchases", 
    "fiscal stability measures", "credit guarantee", "currency devaluation", 
    "trade facilitation", "debt forgiveness", "credit growth measures", "foreign direct investment", 
    "trade stimulus", "income growth", "wealth distribution", "market liquidity", "capital access", 
    "public sector support", "industrial stimulus", "expansionary fiscal policy", 
    "accommodative monetary stance"
    # Add more terms as needed
]

# count the hawkish and dovish word for each text
def count_words(text, word_list):
    count = sum(text.lower().count(word) for word in word_list)
    return count

df['dovish_count'] = df['text'].apply(lambda x: count_words(x, dovish_words))
df['hawkish_count'] = df['text'].apply(lambda x: count_words(x, hawkish_words))

# Clasification according to the count
def classify_text(row):
    if row['dovish_count'] > row['hawkish_count']*1.5:
        return 'dovish'
    elif row['hawkish_count'] > row['dovish_count']*1.5:
        return 'hawkish'
    else:
        return 'neutral'

df['classification_w'] = df.apply(classify_text, axis=1)




In [64]:
df.head()

,date,text,dovish_count,hawkish_count,classification
0,2011-01-26,"The Federal Reserve, the central bank of the U...",7,13,hawkish
1,2011-03-15,"The Federal Reserve, the central bank of the U...",6,17,hawkish
2,2011-04-27,"The Federal Reserve, the central bank of the U...",6,16,hawkish
3,2011-06-22,"The Federal Reserve, the central bank of the U...",5,12,hawkish
4,2011-08-09,"The Federal Reserve, the central bank of the U...",6,12,hawkish


In [65]:
df['classification'] = df.apply(classify_document, axis=1)
classification_counts = df['classification'].value_counts()
print(classification_counts)

Hawkish: 372
Dovish: 350
Neutral: 147


## 2. Factor similarity

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the finbert
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = BertModel.from_pretrained('yiyanghkust/finbert-tone')


# Define expanded hawkish and dovish reference phrases
hawkish_reference = [
    "inflationary pressures, interest rate hikes, tightening of monetary policy",
    "aggressive interest rate increases, combating inflation, controlling price stability",
    "hawkish stance, reducing balance sheet, high inflation concerns",
    "restrictive monetary policy, tightening liquidity, curbing economic overheating",
    "higher borrowing costs, maintaining a strong dollar, inflation targeting",
    "risk of overheating economy, proactive monetary measures, firm interest rate policies",
    "tightening the money supply, signals of future rate hikes, preemptive measures against inflation",
    "fighting inflation, reducing demand, restraining economic growth",
    "stricter lending conditions, focus on price stability, monetary policy normalization",
    "emphasis on inflation control, balancing growth with inflation risks"
]

dovish_reference = [
    "lower rates, economic stimulus, accommodative policy stance",
    "support for economic growth, easing monetary policy, maintaining low-interest rates",
    "encouraging investment, promoting consumer spending, softening credit conditions",
    "stimulus measures, nurturing economic recovery, flexible monetary policy",
    "prolonged low rates, enhancing liquidity, dovish outlook on inflation",
    "supporting job growth, minimizing economic disruption, fostering sustainable growth",
    "favoring accommodative stance, preventing deflationary pressures, emphasis on growth",
    "lowering borrowing costs, prioritizing economic stability, facilitating easy credit",
    "expansionary fiscal policy, encouraging financial markets, bolstering consumer confidence",
    "investing in infrastructure, stimulating demand, creating a supportive economic environment"
]


# Embedding and split the long sentence
def get_sentence_embedding(sentence):
    # Use tokenizer，set truncation=True to split
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # use[CLS] token
    return outputs.last_hidden_state[:, 0, :].numpy()

# Claculating the similarity
def calculate_similarity(text, reference_text):
    text_embedding = get_sentence_embedding(text)
    reference_embedding = get_sentence_embedding(reference_text)
    return cosine_similarity(text_embedding, reference_embedding)[0][0]

# claculate the hawkish and dovish simnilarity for each text
def process_text(text):
    try:
        hawkish_sim = calculate_similarity(text, hawkish_reference)
        dovish_sim = calculate_similarity(text, dovish_reference)
        return pd.Series([hawkish_sim, dovish_sim])
    except Exception as e:
        print(f"Error processing text: {e}")
        return pd.Series([None, None])

# Apply the funtion
df[['hawkish_similarity', 'dovish_similarity']] = df['text'].apply(process_text)

def classify_document(row):
    if pd.isnull(row['hawkish_similarity']) or pd.isnull(row['dovish_similarity']):
        return 'unknown'
    if row['hawkish_similarity'] > row['dovish_similarity']*1.2:
        return 'hawkish'
    elif row['dovish_similarity'] > row['hawkish_similarity']*1.2:
        return 'dovish'
    else:
        return 'neutral'

df['classification_s'] = df.apply(classify_document, axis=1)


In [110]:
df.head()

,date,text,dovish_count,hawkish_count,classification,classification_numeric,hawkish_similarity,dovish_similarity,classification_s
0,2011-01-26,"The Federal Reserve, the central bank of the U...",7,13,hawkish,1,0.449360,0.650187,neutral
1,2011-03-15,"The Federal Reserve, the central bank of the U...",6,17,hawkish,1,0.424052,0.622608,neutral
2,2011-04-27,"The Federal Reserve, the central bank of the U...",6,16,hawkish,1,0.424086,0.621737,neutral
3,2011-06-22,"The Federal Reserve, the central bank of the U...",5,12,hawkish,1,0.503104,0.660841,hawkish
4,2011-08-09,"The Federal Reserve, the central bank of the U...",6,12,hawkish,1,0.540741,0.665799,hawkish


In [106]:
# Count the occurrences of each classification
classification_counts = df['classification_s'].value_counts()

# Print the counts
print(classification_counts)


classification_s
dovish     379
hawkish    353
neutral    137
Name: count, dtype: int64


In [109]:
#Comparison
matching_hawkish = ((df['classification'] == 'hawkish') & (df['classification_s'] == 'hawkish')).sum()
matching_dovish = ((df['classification'] == 'dovish') & (df['classification_s'] == 'dovish')).sum()
matching_neutral = ((df['classification'] == 'neutral') & (df['classification_s'] == 'neutral')).sum()

# Print results
print(f"Hawkish count where both classifications match: {matching_hawkish}")
print(f"Dovish count where both classifications match: {matching_dovish}")
print(f"Neutral count where both classifications match: {matching_neutral}")


Hawkish count where both classifications match: 188
Dovish count where both classifications match: 190
Neutral count where both classifications match: 22


In [111]:
#Save the documents
df.to_csv('FOMC_classification_results.csv', index=False)